# What is this "mystery" 2nd component?
---
What we want to do here:
* Recreate the BPT and WHAV* diagrams
* Interrogate the properties of the "mystery" 2nd component
* Look for correlations between the properties of the "mystery" 2nd component and other quantities 

Random ideas:
* is the 2nd component tied to stellar rotation at all?

In [1]:
%matplotlib widget 

In [2]:
import os, sys 
import numpy as np
import pandas as pd 
from astropy.visualization import hist

from spaxelsleuth.loaddata.sami import load_sami_df
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours, plot2dscatter
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram, vmin_fn, vmax_fn, cmap_fn, label_fn, fname_fn, bpt_labels, bpt_colours, ncomponents_colours, ncomponents_labels, component_colours, component_labels

import matplotlib.pyplot as plt 
plt.ion()
plt.close("all")

In [3]:
savefigs = False

In [4]:
# Load the entire SAMI sample
df_all = load_sami_df(ncomponents="recom",
                  bin_type="default",
                  eline_SNR_min=5,
                  correct_extinction=True,
                  debug=False)

In load_sami_df(): Loading DataFrame...
In load_sami_df(): Finished!


In [5]:
# Limit sample to having high-quality multi-component fits
df = df_all[df_all["Number of components"] == df_all["Number of components (original)"]]
print(f"{df.shape[0] / df_all.shape[0] * 100}% of spaxels have high-quality components only")

61.04448405265259% of spaxels have high-quality components only


In [6]:
# Make a copy containing only SF-like spaxels
df_SF = df.copy()
df_SF = df_SF[df_SF["BPT (total)"] == "SF"]

## Stacked bar chart: # of fitted components by spectral classification

In [28]:
counts_0 = []
counts_1 = []
counts_2 = []
counts_3 = []
for bpt in bpt_labels:
    cond = df["BPT (total)"] == bpt
    cond_0 = cond & (df["Number of components"] == 0)
    cond_1 = cond & (df["Number of components"] == 1)
    cond_2 = cond & (df["Number of components"] == 2)
    cond_3 = cond & (df["Number of components"] == 3)
    counts_0.append(df[cond_0].shape[0] / df[cond].shape[0] * 100)
    counts_1.append(df[cond_1].shape[0] / df[cond].shape[0] * 100)
    counts_2.append(df[cond_2].shape[0] / df[cond].shape[0] * 100)
    counts_3.append(df[cond_3].shape[0] / df[cond].shape[0] * 100)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7, 5))
ax.bar(bpt_labels, counts_0, width=0.6, label=ncomponents_labels[0] + " components", color=ncomponents_colours[0])
ax.bar(bpt_labels, counts_1, width=0.6, bottom=counts_0, label=ncomponents_labels[1] + " component", color=ncomponents_colours[1])
ax.bar(bpt_labels, counts_2, width=0.6, bottom=np.array(counts_0) + np.array(counts_1), label=ncomponents_labels[2] + " components", color=ncomponents_colours[2])
ax.bar(bpt_labels, counts_3, width=0.6, bottom=np.array(counts_0) + np.array(counts_1) + np.array(counts_2), label=ncomponents_labels[3] + " components", color=ncomponents_colours[3])
ax.grid()
ax.set_ylim([0, 105])
ax.set_ylabel("Percentage")
ax.set_xlabel("Spectral classification")
ax.legend(loc="lower center", bbox_to_anchor=(0.5, 1.02), ncol=2, fontsize="small")
plt.xticks(rotation=90)

if savefigs:
    fname = os.path.join(fig_path, f"bar_ncomponents_by_spectral_class.pdf")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    print(f"File saved at: {fname}")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Histograms showing the distribution in various quantities in components 1, 2 and 3

In [11]:
for col in ["sigma_gas", "v_gas", "log HALPHA EW"]:
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    for nn in range(1, 4):
        hist(df[f"{col} (component {nn})"], range=(vmin_fn(col), vmax_fn(col)), bins="scott", histtype="step", color=component_colours[nn - 1], ax=axs[0], normed=True, label=f"Component {nn}")
        hist(df_SF[f"{col} (component {nn})"], range=(vmin_fn(col), vmax_fn(col)), bins="scott", histtype="step", color=component_colours[nn - 1], ax=axs[1], normed=True, label=f"Component {nn}")
    axs[0].set_xlabel(label_fn(col))
    axs[1].set_xlabel(label_fn(col))
    axs[0].set_ylabel("N (normalised)")
    axs[1].legend()
    axs[0].set_title("All spaxels")
    axs[1].set_title("Star-forming spaxels only")
    axs[0].grid()
    axs[1].grid()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.close("all")

In [26]:
# Histogram showing the offset in velocity from component 1 of components 2 and 3
col = "delta v_gas"
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 4))
hist(df[f"{col} (2/1)"], range=(-300, +300), bins="scott", histtype="step", color="black", ax=ax, normed=False, alpha=0.5, label="Full sample")
hist(df_SF[f"{col} (2/1)"], range=(-300, +300), bins="scott", histtype="step", color="blue", ax=ax, normed=False, label="Star-forming spaxels")
ax.grid()
ax.set_ylabel("N")
ax.set_xlabel(r"$\Delta v_{\rm gas}$ (2/1)")
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## What happens if we plot the gas velocity of component 2 against that of component 1?

In [7]:
# Experimenting: 2D histogram 
fig = plot2dhistcontours(df_SF,
                         col_x="v_gas (component 1)",
                         col_y="v_gas (component 2)",
                         col_z="count", log_z=True, 
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         nbins=50)
ax = fig.get_axes()[0]
ax.grid()
ax.set_xlabel(ax.get_xlabel() + " (component 1)")
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# Experimenting: 2D histogram 
fig = plot2dhistcontours(df_SF,
                         col_x="v_gas (component 1)",
                         col_y="v_gas (component 2)",
                         col_z="v_gas - v_* (component 2)",
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         vmin=-100, vmax=+100,
                         nbins=50)
ax = fig.get_axes()[0]
ax.grid()
ax.set_xlabel(ax.get_xlabel() + " (component 1)")
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
for nn in range(1, 4):
    fig = plot2dhistcontours(df_SF,
                             col_x="v_*",
                             col_y=f"v_gas (component {nn})",
                             col_z="count", log_z=True, 
                             # col_z=f"sigma_*",
                             xmin=-300, xmax=300, 
                             ymin=-300, ymax=300, 
                             nbins=50)
    ax = fig.get_axes()[0]
    ax.grid()
    ax.set_ylabel(ax.get_ylabel() + f" (component {nn})")
    ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")

/home/u5708159/python/Modules/spaxelsleuth/plotting/plotgalaxies.py:472: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Nice examples of galaxies with "hysteresis" 
* 301346
* 619095
* 41144
* 460374
* 84107 (not hysteresis, but rather no velocity offset from component 1 at all)
* 272990


In [26]:
# Re-make the above plots but overlaid with points from individual galaxies 
fig = plot2dhistcontours(df_SF,
                         col_x="v_gas (component 1)",
                         col_y="v_gas (component 2)",
                         col_z="count", log_z=False, cmap="gray_r", alpha=0.4,
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         plot_colorbar=False,
                         nbins=50)
ax = fig.get_axes()[0]
 
gal = 460374
df_gal = df_all[df_all["catid"] == gal]
_ = plot2dscatter(df_gal, col_x="v_gas (component 1)", col_y="v_gas (component 2)", col_z="sigma_gas (component 2)",
                  xmin=-300, xmax=300, ymin=-300, ymax=300, vmax=150,
                  plot_colorbar=True,
                  ax=ax)

# Decorations 
ax.grid()
ax.set_xlabel(ax.get_xlabel() + " (component 1)")
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")
ax.plot([-300, 300], [0, 0], color="k")
ax.plot([0, 0], [-300, 300], color="k")

# SDSS image
from spaxelsleuth.plotting.sdssimg import plot_sdss_image
plot_sdss_image(df_gal)

/home/u5708159/python/Modules/spaxelsleuth/plotting/plotgalaxies.py:472: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/sdssimg.py:164: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(nrows=1, ncols=1, figsize=figsize, subplot_kw={"projection": wcs})


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## How to tell whether the 2nd component is due to residuals in the stellar template fitting?

In [10]:
# Experimenting: 2D histogram 
fig = plot2dhistcontours(df_SF,
                         col_x="v_gas - v_* (component 2)",
                         col_y="sigma_gas - sigma_* (component 2)",
                         col_z="count", log_z=True,
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         nbins=100)
ax = fig.get_axes()[0]
ax.grid()
ax.set_xlabel(ax.get_xlabel() + " (component 2)")
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## BPT diagrams
---

In [20]:
col_z = "log sigma_gas"
for nn in range(1, 4):
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)

    # Plot 2D histograms of the subset
    plot2dhistcontours(df, col_x="log N2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=np.log10(20), vmax=np.log10(150), cmap="Spectral_r", ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log S2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=np.log10(20), vmax=np.log10(150), cmap="Spectral_r", ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log O1 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=np.log10(20), vmax=np.log10(150), cmap="Spectral_r", ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"BPT_SAMI_{fname_fn(col_z)}_component{nn}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")

col_z = "sigma_gas - sigma_*"
for nn in range(1, 4):
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)

    # Plot 2D histograms of the subset
    plot2dhistcontours(df, col_x="log N2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=-200, vmax=+200, ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log S2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=-200, vmax=+200, ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log O1 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=-200, vmax=+200, ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"BPT_SAMI_{fname_fn(col_z)}_component{nn}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## WHAV* diagram
---

In [15]:
col_z_list = ["count", "BPT (numeric) (total)", "v_grad", "r/R_e"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for nn in range(1, 4):
        plot2dhistcontours(df, col_x=f"sigma_gas - sigma_* (component {nn})",
                           col_y=f"log HALPHA EW (component {nn})",
                           col_z=f"{col_z} (component {nn})" if f"{col_z} (component {nn})" in df else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[nn - 1], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if nn == 3 - 1 else False)
        # Decorations
        axs[nn - 1].grid()
        axs[nn - 1].set_ylabel("") if nn > 0 else None
        axs[nn - 1].set_yticklabels([]) if nn > 0 else None
        axs[nn - 1].text(s=f"Component {nn}", x=0.05, y=0.95, transform=axs[nn - 1].transAxes, verticalalignment="top")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"WHAV_nostar_SAMI_{fname_fn(col_z)}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")        
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …